# Yelp API - Lab



## Introduction 

Now that we've seen how the Yelp API works, it's time to put those API and SQL skills to work in order to do some basic business analysis! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to:
* Create a DB on AWS to store information from Yelp about businesses
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and insert the information into your DB
* Perform pagination to retrieve troves of data!
* Write SQL queries to answer questions about your data 

## Problem Introduction

For this lab you will analyze the yelp data for a group of businesses to learn more about an industry. You will choose a type of business (Italian Restuarants, Nail Salons, Crossfit gyms) and a location to analyze. Then you will get data from the Yelp API, store that data in a SQL Database on AWS, and write queries to answer questions about the data. 


### Process:

1. Read through the SQL questions and the API documentation to determine which pieces of information you need to pull from the Yelp API.

2. Create a DB schema with 2 tables. One for the businesses and one for the reviews.

3. Create code to:
  - Perform a search of businesses using pagination
  - Parse the API response for specific data points
  - Insert the data into your AWS DB

4. Use the functions above in a loop that will paginate over the results to retrieve all of the results. 

5. Create functions to:
  - Retrieve the reviews data of one business
  - Parse the reviews response for specific review data
  - Insert the review data into the DB

6. Using SQL, query all of the business IDs. Using the 3 Python functions you've created, run your business IDs through a loop to get the reviews for each business and insert them into your DB.

7. Write SQL queries to answer the following questions about your data.


Bonus Steps:  
- Place your helper functions in a package so that your final notebook only has the major steps listed.
- Rewrite your business search functions to be able take an argument for the type of business you are searching for.
- Add another group of businesses to your database.


 
## SQL Questions:

- Which are the 5 most reviewed businesses?
- What is the highest rating recieved in your data set and how many businesses have that rating?
- What percentage of businesses have a rating greater than or  4.5?
- What percentage of businesses have a rating less than 3?
- What is the average rating of restaurants that have a price label of one dollar sign? Two dollar signs? Three dollar signs? 
- Return the text of the reviews for the most reviewed restaurant. 
- Return the name of the business with the most recent review. 
- Find the highest rated business and return text of the most recent review. If multiple business have the same rating, select the restaurant with the most reviews. 
- Find the lowest rated business and return text of the most recent review.  If multiple business have the same rating, select the restaurant with the least reviews. 


## Part I - Set up the DB

Start by reading SQL questions above to get an understanding of the data you will need. Then, read the documentation of Yelp API to understand what data you will receive in the response.  


Now that you are familiar with the data, create your SQL queries to create the DB and the appropriate tables. 

In [23]:
## Connect to DB server on AWS
import mysql.connector
import config
import requests

In [24]:
cnx = mysql.connector .connect(
    host = config.host,
    user = config.user,
    passwd = config.password
)
cursor = cnx.cursor()

In [25]:
## Create new DB
from mysql.connector import errorcode
db_name = 'lunch'
def create_database(cursor, database):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

try:
    cursor.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

In [26]:
# Create a table for the Businesses
DB_NAME = 'lunch'

# using dictionary to save CREATE TABLE statements
TABLES = {}
TABLES['businesses'] = (
    """CREATE TABLE businesses (
       business_id varchar(50) NOT NULL,
       name varchar(50) NOT NULL,
       price varchar(5) NOT NULL,
       categories varchar(50) NOT NULL,
       location varchar(50) NOT NULL,
       review_count int(10) NOT NULL,
       rating float(5) NOT NULL,
       PRIMARY KEY (business_id)
     ) ENGINE=InnoDB""")

In [27]:
# Create a table for the reviews
TABLES['reviews'] = (
    """CREATE TABLE reviews (
       review_id varchar(50) NOT NULL,
       business_id varchar(50) NOT NULL,
       text varchar(1000) NOT NULL,
       time_created varchar(50) NOT NULL,
       PRIMARY KEY (review_id)
     ) ENGINE=InnoDB""")

In [28]:
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

cursor.close()
cnx.close()

Creating table businesses: already exists.
Creating table reviews: already exists.


## Part 2: Create ETL pipeline for the business data from the API

In [56]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = db_name,
    buffered = True
)
cursor = cnx.cursor()

In [ ]:
# Sample code
def yelp_call(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    data = response.json()['businesses']
    return data

In [45]:
# Write a function to make a call to the yelp API
# url_params --> location = "Financial District, Manhattan, NY"
def call_yelp(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    data = response.json()['businesses']
    return data

In [46]:
import api # contains api key

In [47]:
test = call_yelp({'location': 'Financial District, Manhattan, NY', 'categories': 'food'}, api.key)

In [49]:
test[0]

{'id': 'rQpO1Eie_zmeAOQ9s-T0sQ',
 'alias': 'hole-in-the-wall-new-york-2',
 'name': 'Hole in the Wall',
 'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/KQwNs2HFWF0GDEGoSYgAEg/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/hole-in-the-wall-new-york-2?adjust_creative=ih6A7b6xiJrw8qxUMyi_ZA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=ih6A7b6xiJrw8qxUMyi_ZA',
 'review_count': 500,
 'categories': [{'alias': 'breakfast_brunch', 'title': 'Breakfast & Brunch'},
  {'alias': 'cocktailbars', 'title': 'Cocktail Bars'},
  {'alias': 'juicebars', 'title': 'Juice Bars & Smoothies'}],
 'rating': 4.0,
 'coordinates': {'latitude': 40.70837, 'longitude': -74.00563},
 'transactions': [],
 'price': '$$',
 'location': {'address1': '15 Cliff St',
  'address2': '',
  'address3': '',
  'city': 'New York',
  'zip_code': '10038',
  'country': 'US',
  'state': 'NY',
  'display_address': ['15 Cliff St', 'New York, NY 10038']},
 'phone': '+12126029991',
 'display_phone

In [55]:
# TABLES['businesses'] = (
#     """CREATE TABLE businesses (
#        business_id varchar(50) NOT NULL,
#        name varchar(50) NOT NULL,
#        price varchar(5) NOT NULL,
#        categories varchar(50) NOT NULL,
#        location varchar(50) NOT NULL,
#        review_count int(10) NOT NULL,
#        rating float(5) NOT NULL,
#        PRIMARY KEY (business_id)
#      ) ENGINE=InnoDB""")
for index in range(0, len(test)):
    print(test[index]['id'])
    print(test[index]['name'])
    print(test[index]['price'])
    print(test[index]['categories'])
    print(test[index]['location']['display_address'])
    print(test[index]['review_count'])
    print(test[index]['rating'])

rQpO1Eie_zmeAOQ9s-T0sQ
Hole in the Wall
$$
[{'alias': 'breakfast_brunch', 'title': 'Breakfast & Brunch'}, {'alias': 'cocktailbars', 'title': 'Cocktail Bars'}, {'alias': 'juicebars', 'title': 'Juice Bars & Smoothies'}]
['15 Cliff St', 'New York, NY 10038']
500
4.0
pL2tig3FxJcrZ4wDIO4TsA
Eataly NYC Downtown
$$
[{'alias': 'grocery', 'title': 'Grocery'}, {'alias': 'italian', 'title': 'Italian'}, {'alias': 'meats', 'title': 'Meat Shops'}]
['101 Liberty St', 'Fl 3', 'New York, NY 10006']
906
3.5
o6q3jm-dU5A6nV3r2lBg9A
Chinatown Ice Cream Factory
$
[{'alias': 'icecream', 'title': 'Ice Cream & Frozen Yogurt'}, {'alias': 'desserts', 'title': 'Desserts'}]
['65 Bayard St', 'New York, NY 10013']
2688
4.0
t4B9GkAuiYIjFCt9SlK3EQ
Poke Bowl
$$
[{'alias': 'poke', 'title': 'Poke'}, {'alias': 'sushi', 'title': 'Sushi Bars'}]
['104 Fulton St', 'New York, NY 10038']
304
4.0
qzdCvtgpCDjNVV_zRJB5EQ
Voyager Espresso
$
[{'alias': 'coffee', 'title': 'Coffee & Tea'}, {'alias': 'cafes', 'title': 'Cafes'}, {'alias

In [59]:
# write a function to parse the API response 
# so that you can easily insert the data in to the DB
def parse_yelp(data):
    businesses = []
    for index in range(0, len(data)):
        business_i = data[index]
        business_info = (business_i['id'], business_i['name'], business_i['price'], 
                         business_i['categories'], business_i['location']['display_address'], 
                         business_i['review_count'], business_i['rating'])
        businesses.append(business_info)
    return businesses

In [61]:
# Write a function to take your parsed data and insert it into the DB
test_data = parse_yelp(test)

insert_statement = "INSERT INTO businesses (business_id, name, price, categories, location, review_count, rating) VALUES (%s, %s, %s, %s, %s, %s, %s)"
cursor.executemany(insert_statement, test_data)
cnx.commit()

InterfaceError: Failed executing the operation; Python type list cannot be converted

In [ ]:
# Write a script that combines the three functions above into a single process.

## Part 3: Create ETL pipeline for the restaurant review data from the API

In [ ]:
# write a SQL query to pull back all of the business ids 
# you will need these ids to pull back the reviews for each restaurant

In [ ]:
# write a function that takes a business id 
# and makes a call to the API for reivews


In [ ]:
# Write a function to parse out the relevant information from the reviews

In [ ]:
# write a function to insert the parsed data into the reviews table

In [ ]:
# combine the functions above into a single script  

## Part 4: Write SQL queries that will answer the questions posed. 

In [ ]:
# create connection

In [ ]:
# execute SQL queries

# Extra Reference help

###  Pagination

Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. 

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and APIs can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

***Below is sample code that you can use to help you deal with the pagination parameter and bring all of the functions together.***


***Also, something might cause your code to break while it is running. You don't want to constantly repull the same data when this happens, so you should insert the data into the database as you call and parse it, not after you have all of the data***


In [ ]:
def yelp_call(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    
    data = response.json()['businesses']
    return data

In [ ]:
def parse_results(results):
    
    return parsed_results

In [ ]:
# Your code here; use a function or loop to retrieve all the results from your original request
import time



def all_results(url_params, api_key):
    num = response.json()['total']
    print('{} total matches found.'.format(num))
    cur = 0
    
    while cur < num and cur < 1000:
        url_params['offset'] = cur
        results = yelp_call(url_params, api_key)
        parsed_results = parse_results(results)
        db_insert(parsed)
        time.sleep(1) #Wait a second
        cur += 50

term = 'pizza'
location = 'Astoria NY'
url_params = {  'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit' : 50
             }
df = all_results(url_params, api_key)
print(len(df))
df.head()

### Sample SQL Query 

Below is a SQL query to create a table.  Additionally here is a link to create a table with a foreign key.

http://www.mysqltutorial.org/mysql-foreign-key/

```CREATE TABLE IF NOT EXISTS tasks (
    task_id INT AUTO_INCREMENT,
    title VARCHAR(255) NOT NULL,
    start_date DATE,
    due_date DATE,
    status TINYINT NOT NULL,
    priority TINYINT NOT NULL,
    description TEXT,
    PRIMARY KEY (task_id)
)  ENGINE=INNODB;```

### Using DB:
    
For this lab, you can either store the data on one DB or put in on both of the partners DBs. If you decide to put it on one DB, you want to make sure both partners have access to it.  To do this you want to add a user to your DB.  

[how to add a new user](https://howchoo.com/g/mtm3zdq2nzv/how-to-add-a-mysql-user-and-grant-privileges)